In [ ]:
import requests, bs4
import pandas as pd
# Вводим тикер актива и даты начала и конца.
TICKER = 'SBER'
DATE_FROM = '2013-03-25'
DATE_TILL = '2018-08-15'
start = 0
hlocv = []
while True:
    # Собираем информацию об инструменте, для запроса рыночных данных.
    ticker_info = requests.get('http://iss.moex.com/iss/securities.xml?q=%s' % (TICKER)).text
    ticker_info = bs4.BeautifulSoup(ticker_info, 'html.parser') 
    ticker_info = ticker_info.find('rows').find('row', {'secid': '%s' % (TICKER)})
    engine = ticker_info.get('group').split('_')[0]
    market = ticker_info.get('group').split('_')[1]
    board = ticker_info.get('primary_boardid')
    # Запрашиваем маркет дату.
    items = requests.get('http://iss.moex.com/iss/history/engines/%s/markets/%s/boards/%s/securities/%s/candles.xml?from=%s&till=%s&interval=0&start=%d' % (engine, market, board, TICKER, DATE_FROM, DATE_TILL, start)).text
    items = bs4.BeautifulSoup(items, 'html.parser') 
    items = items.find('rows').find_all('row')
    # Составляем словарь из параметров торгового дня и формируем один большой список из всех дней диапазона дат.
    for i in items:
        date = i.get('tradedate')
        open_p = i.get('open')
        high = i.get('high')
        low = i.get('low')
        close = i.get('close')
        volume = i.get('volume')
        hlocv.append({'date': date, 'open': open_p, 'high': high, 'low': low, 'close': close, 'volume': volume })
    # Проверяем является ли дата последнего торгового дня в выдаче конечной датой нашего интервала.    
    if items[-1].get('tradedate') != DATE_TILL:
        # Если нет, то загружаем еще
        start +=len(items)
    else:
        break
# формируем датафрейм
df = pd.DataFrame(hlocv, columns = ['date','open', 'high', 'low', 'close', 'volume'])
df